In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.parse import urlparse
import os
import time
import re

In [ ]:
# URL страницы для парсинга
url = "http://www.cbr.ru/Crosscut/LawActs/Page/94917?Date.Time=Any&Page=0"

# папка для сохранения скачанных документов
download_folder = "documents"
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Определение заголовков для имитации браузера (чтобы избежать блокировки)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    'Accept-Language': 'en-US,en;q=0.5'
}

page_number = 0

# Функция для скачивания документа
def download_document(url, folder):
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status() 
    
        # Получаем расширение файла из заголовка "Content-Type"
        content_type = response.headers.get('Content-Type')
        file_extension = '.pdf' if 'pdf' in content_type else '.md'

        if '?' in url:
            url = url[:url.find('?')]
    
        file_name = os.path.join(folder, f"{url.split('/')[-1]}{file_extension}")
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"Документ скачан: {file_name}")
        return True
    except Exception as e:
        print(f"Ошибка при скачивании документа: {e}")
        return False

# Основной цикл для парсинга страницы
while page_number<297:
    print(url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Находим все элементы <a> с нужными документами
    document_links = soup.select('.title-source.offset-md-4 .title a, .title-source.offset-md-4 .title span.cross-hidden + a')

    # Скачиваем каждый документ
    for link in document_links:
        if 'file' not in link['href'].lower():
            continue
        document_url = urljoin("http://www.cbr.ru", link['href'])
        download_document(document_url, download_folder)
        time.sleep(4)

    url = "http://www.cbr.ru/Crosscut/LawActs/Page/94917"
    page_number += 1
    url = urljoin(url, "?Date.Time=Any&Page=" + str(page_number))
    
print("Все документы скачены")